<a href="https://colab.research.google.com/github/mohsenhariri/blog/blob/main/nnunet2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import os
import shutil
from collections import OrderedDict

import json
import matplotlib.pyplot as plt
import nibabel as nib

import numpy as np
import torch



!pip install SimpleITK==2.0.2

!pip install nnunet 

import nnunet



from google.colab import drive
drive.mount('/content/drive',force_remount = True)

drive_dir = "/content/drive/My Drive"
mount_dir = os.path.join(drive_dir, "Colab Notebooks")
base_dir = os.getcwd()


assert os.path.exists(drive_dir) # if this fails, something went wrong with mounting GoogleDrive
if os.path.exists(mount_dir) is False:
    os.makedirs(mount_dir)


def make_if_dont_exist(folder_path,overwrite=False):
    """
    creates a folder if it does not exists
    input: 
    folder_path : relative path of the folder which needs to be created
    over_write :(default: False) if True overwrite the existing folder 
    """
    if os.path.exists(folder_path):
        
        if not overwrite:
            print(f"{folder_path} exists.")
        else:
            print(f"{folder_path} overwritten")
            shutil.rmtree(folder_path)
            os.makedirs(folder_path)

    else:
      os.makedirs(folder_path)
      print(f"{folder_path} created!")

     |████████████████████████████████| 47.4 MB 1.7 MB/s 
     |████████████████████████████████| 251 kB 5.1 MB/s 
     |████████████████████████████████| 151 kB 42.9 MB/s 
     |████████████████████████████████| 57 kB 4.8 MB/s 
     |████████████████████████████████| 96 kB 5.4 MB/s 
     |████████████████████████████████| 2.0 MB 61.3 MB/s 
  Created wheel for nnunet: filename=nnunet-1.7.0-py3-none-any.whl size=487821 sha256=d64a6ac67e3ee333a3337b0d2ae89bc92ce7bed2f46117413cf873746c7c7adf
  Stored in directory: /root/.cache/pip/wheels/66/f6/19/c51389976a9ae30212178d1c41497893504b900f043fe1c665
  Created wheel for batchgenerators: filename=batchgenerators-0.23-py3-none-any.whl size=84779 sha256=37de2baf7305d1b2d4acffb9c4e232af8519e9660a2d1420b5b2931fc24d67d2
  Stored in directory: /root/.cache/pip/wheels/81/3d/a2/9a602ce56458c684d2962ec89eddd6bc06b00687735e2dd11a
  Created wheel for dicom2nifti: filename=dicom2nifti-2.3.2-py3-none-any.whl size=44320 sha256=727cb62123656ae2bca893f88eb02a



Please cite the following paper when using nnUNet:

Isensee, F., Jaeger, P.F., Kohl, S.A.A. et al. "nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation." Nat Methods (2020). https://doi.org/10.1038/s41592-020-01008-z


If you have questions or suggestions, feel free to open an issue at https://github.com/MIC-DKFZ/nnUNet

Mounted at /content/drive


# New Section

# New Section

In [4]:
# Maybe move path of preprocessed data directly on content - this may be signifcantely faster!
print("Current Working Directory {}".format(os.getcwd()))
path_dict = {
    "nnUNet_raw_data_base" : os.path.join(mount_dir, "nnUNet_raw_data_base"), 
    "nnUNet_preprocessed" : os.path.join(mount_dir, "nnUNet_preprocessed"), # 1 experiment: 1 epoch took 112s
    # "nnUNet_preprocessed" : os.path.join(base_dir, "nnUNet_preprocessed"), # 1 experiment: 1 epoch took 108s -> seems faster take this
    "RESULTS_FOLDER" : os.path.join(mount_dir, "nnUNet_Results_Folder"),
    "RAW_DATA_PATH" : os.path.join(mount_dir, "RawData"), # This is used here only for convenience (not necessary for nnU-Net)!
}

# Write paths to environment variables
for env_var, path in path_dict.items():
  os.environ[env_var] = path 

# Check whether all environment variables are set correct!
for env_var, path in path_dict.items():
  if os.getenv(env_var) != path:
    print("Error:")
    print("Environment Variable {} is not set correctly!".format(env_var))
    print("Should be {}".format(path))
    print("Variable is {}".format(os.getenv(env_var)))
  make_if_dont_exist(path, overwrite=False)

print("If No Error Occured Continue Forward. =)")

Current Working Directory /content
/content/drive/My Drive/Colab Notebooks/nnUNet_raw_data_base exists.
/content/drive/My Drive/Colab Notebooks/nnUNet_preprocessed exists.
/content/drive/My Drive/Colab Notebooks/nnUNet_Results_Folder exists.
/content/drive/My Drive/Colab Notebooks/RawData exists.
If No Error Occured Continue Forward. =)


In [5]:
!nnUNet_download_pretrained_model Task001_BrainTumour 



Please cite the following paper when using nnUNet:

Isensee, F., Jaeger, P.F., Kohl, S.A.A. et al. "nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation." Nat Methods (2020). https://doi.org/10.1038/s41592-020-01008-z


If you have questions or suggestions, feel free to open an issue at https://github.com/MIC-DKFZ/nnUNet


######################################################
!!!!!!!!!!!!!!!!!!!!!!!!WARNING!!!!!!!!!!!!!!!!!!!!!!!
######################################################
Using the pretrained model weights is subject to the license of the dataset they were trained on. Some allow commercial use, others don't. It is your responsibility to make sure you use them appropriately! Use nnUNet_print_pretrained_model_info(task_name) to see a summary of the dataset and where to find its license!
######################################################

Download finished. Extracting...
Done


In [6]:
!ls drive/MyDrive/'Colab Notebooks'/nnUNet_raw_data_base/nnUNet_raw_data/Task001_BrainTumour/imagesTs

BRATS_001_0000.nii.gz  BRATS_001_0002.nii.gz
BRATS_001_0001.nii.gz  BRATS_001_0003.nii.gz


In [7]:
!nnUNet_predict -i drive/MyDrive/'Colab Notebooks'/nnUNet_raw_data_base/nnUNet_raw_data/Task001_BrainTumour/imagesTs -o OUTPUT_DIRECTORY -t Task001_BrainTumour




Please cite the following paper when using nnUNet:

Isensee, F., Jaeger, P.F., Kohl, S.A.A. et al. "nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation." Nat Methods (2020). https://doi.org/10.1038/s41592-020-01008-z


If you have questions or suggestions, feel free to open an issue at https://github.com/MIC-DKFZ/nnUNet

using model stored in  /content/drive/My Drive/Colab Notebooks/nnUNet_Results_Folder/nnUNet/3d_fullres/Task001_BrainTumour/nnUNetTrainerV2__nnUNetPlansv2.1
This model expects 4 input modalities for each image
Found 1 unique case ids, here are some examples: ['BRATS_001']
If they don't look right, make sure to double check your filenames. They must end with _0000.nii.gz etc
number of cases: 1
number of cases that still need to be predicted: 1
emptying cuda cache
loading parameters for folds, None
folds is None so we will automatically look for output folders (not using 'all'!)
found the following folds:  ['/content/drive/My Drive/C